# Bing Grounding Integration with Azure AI Foundry Agents

This notebook demonstrates how to integrate **Grounding with Bing Search** into Azure AI Foundry agents for real-time web search capabilities.

## What is Bing Grounding?

**Bing Grounding** is an Azure-provided tool that enables agents to:
- Search the web in real-time using Bing Search API
- Retrieve current information, news, and data
- Provide cited sources and references
- Ground responses in factual, up-to-date web content

## Key Benefits

✅ **Real-time Information**: Access current data and recent events  
✅ **Automatic Execution**: Azure handles all tool calls automatically  
✅ **Built-in Citations**: Responses include source links  
✅ **Easy Integration**: Simple SDK-based setup  
✅ **Production Ready**: Managed by Azure infrastructure  

## Use Cases

- **Research Assistants**: Find current information and industry trends
- **News Bots**: Retrieve latest headlines and breaking news
- **Customer Support**: Access product information and reviews
- **Data Analysts**: Gather real-time statistics and reports
- **Knowledge Workers**: Research topics with credible sources

## Prerequisites

Before starting, ensure you have:
1. Azure AI Foundry project created
2. Bing Search resource connected to your project
3. Connection ID for your Bing resource
4. Azure credentials configured (Azure CLI or DefaultAzureCredential)

---

## Table of Contents

1. [Part 1: Setup and Configuration](#part-1-setup-and-configuration)
   - Step 1: Import Dependencies and Initialize Client
2. [Part 2: Understanding Bing Grounding Integration](#part-2-understanding-bing-grounding-integration)
   - Integration Architecture
   - Integration Flow
   - Bing Tool vs Custom Functions
3. [Part 3: Initialize Bing Grounding Tool](#part-3-initialize-bing-grounding-tool)
   - Step 1: Import BingGroundingTool
   - Step 2: Initialize with Connection ID
   - Step 3: Inspect Tool Definitions
4. [Part 4: Create Agent with Bing Grounding](#part-4-create-agent-with-bing-grounding)
5. [Part 5: Test the Agent](#part-5-test-the-agent)
   - Run a Simple Query
6. [Part 6: Display Citations (Bing Requirements)](#part-6-display-citations-bing-requirements)
7. [Part 7: Additional Examples](#part-7-additional-examples)
   - Example 1: Different Query Types
8. [Part 8: Cleanup](#part-8-cleanup)
9. [Summary and Best Practices](#summary-and-best-practices)

---

In [ ]:
import os
import shutil

new_path_entry = "/opt/homebrew/bin"  # Replace with the directory you want to add
current_path = os.environ.get('PATH', '')

if new_path_entry not in current_path.split(os.pathsep):
    os.environ['PATH'] = new_path_entry + os.pathsep + current_path
    print(f"Updated PATH for this session: {os.environ['PATH']}")
else:
    print(f"PATH already contains {new_path_entry}: {current_path}")

# You can then verify with shutil.which again
print(f"Location of 'az' found by kernel now: {shutil.which('az')}")

## Part 1: Setup and Configuration

### Step 1: Import Dependencies and Initialize Client

In [ ]:
import os
import sys
from dotenv import load_dotenv
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential

sys.path.append(os.path.join(os.path.dirname(os.getcwd()), 'utils'))
from agent_utils import AgentManager

# Load environment variables
load_dotenv()

# Get project endpoint
endpoint = os.getenv("AZURE_AI_PROJECT_ENDPOINT")

if not endpoint:
    raise ValueError("Please set AZURE_AI_PROJECT_ENDPOINT in .env.local")

# Initialize Azure AI Project Client
project_client = AIProjectClient(
    endpoint=endpoint,
    credential=DefaultAzureCredential()
)
agent_manager = AgentManager(project_client)

print("✅ Setup Complete")
print(f"   Project Client: {project_client}")

---

## Part 2: Understanding Bing Grounding Integration

### Integration Architecture

```
┌─────────────────────────────────────────────────────────────────┐
│                     Your Application Code                       │
├─────────────────────────────────────────────────────────────────┤
│  1. Import BingGroundingTool                                    │
│  2. Initialize with Connection ID                               │
│  3. Pass tool.definitions to agent_manager.create_agent()       │
└────────────────────┬────────────────────────────────────────────┘
                     │
                     ▼
┌─────────────────────────────────────────────────────────────────┐
│                   AgentManager (agent_utils.py)                 │
├─────────────────────────────────────────────────────────────────┤
│  • Receives tools parameter                                     │
│  • Creates ToolSet()                                            │
│  • Extracts tool.definitions                                    │
│  • Combines with custom functions (if any)                      │
│  • Passes toolset.definitions to Azure API                      │
└────────────────────┬────────────────────────────────────────────┘
                     │
                     ▼
┌─────────────────────────────────────────────────────────────────┐
│              Azure AI Foundry Agent Service                     │
├─────────────────────────────────────────────────────────────────┤
│  • Receives agent with Bing tool definitions                    │
│  • When user queries, LLM decides if Bing search is needed      │
│  • Calls Bing API via your connection                           │
│  • Returns results with citations                               │
└─────────────────────────────────────────────────────────────────┘
```

### Integration Flow

1. **Connection Setup**: Configure Bing resource in Azure AI Foundry (one-time)
2. **Tool Initialization**: Import and instantiate `BingGroundingTool` in your code
3. **Agent Creation**: Pass tool definitions to agent
4. **Runtime Execution**: Azure automatically handles all Bing API calls
5. **Response with Citations**: Agent returns results with source links

### Bing Tool vs Custom Functions

Understanding the key differences:

| Aspect | Custom Python Functions | Bing Grounding Tool |
|--------|------------------------|---------------------|
| **Definition Source** | Python callables you provide | BingGroundingTool class (Azure SDK) |
| **Execution Location** | Your application (local/server) | Azure cloud (Bing API) |
| **Connection Required** | No | Yes (`connection_id` required) |
| **Code You Write** | `def my_function(args): ...` | Just pass `bing_tool.definitions` |
| **Tool Schema** | Auto-generated from Python signature | Pre-defined by Azure |
| **Function Execution** | Manual in `run_agent()` loop | Automatic by Azure |
| **Response Handling** | You handle results and return to agent | Azure handles everything |
| **Example Use Cases** | `get_weather()`, `calculate_sum()` | Web search, current events |

**Key Advantage**: With Bing Grounding, Azure manages the entire execution lifecycle - you just configure and forget!

---

## Part 3: Initialize Bing Grounding Tool

### Step 1: Import BingGroundingTool

In [ ]:
from azure.ai.agents.models import BingGroundingTool

print("✅ Imported BingGroundingTool from azure.ai.agents.models")
print(f"   Tool class: {BingGroundingTool}")
print(f"   Module: {BingGroundingTool.__module__}")

### Step 2: Initialize with Connection ID

Get your Bing connection ID from Azure AI Foundry:
1. Go to your Azure AI Foundry project
2. Navigate to **Connected resources**
3. Find your Bing/Grounding resource
4. Copy the connection ID

Format: `/subscriptions/{subscription-id}/resourceGroups/{rg}/providers/.../connections/{connection-name}`

In [ ]:
from dotenv import load_dotenv

# Load environment variables
load_dotenv("../.env")

# Get Bing connection ID from environment
bing_connection_id = os.getenv("BING_CONNECTION_ID")

if not bing_connection_id:
    raise ValueError("Please set BING_CONNECTION_ID in .env file")

# Initialize the Bing Grounding tool
bing_tool = BingGroundingTool(connection_id=bing_connection_id)

print("✅ Initialized BingGroundingTool")
print(f"   Connection ID: {bing_connection_id}")
print(f"   Tool instance: {bing_tool}")

---

## Part 4: Create Agent with Bing Grounding

Now we'll create a Web Research Assistant agent with Bing search capabilities.

In [ ]:
bing_agent = agent_manager.create_agent(
    model=os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME", "gpt-4o"),
    name="Web Research Assistant",
    instructions="""You are a Web Research Assistant with real-time web search capabilities.

## Role
You help users find current information from the web, including:
- Latest news and current events
- Recent industry updates and trends
- Real-time data and statistics
- Current product information and reviews

## Guidelines
1. When asked about current events or recent information, use the Bing search tool to find up-to-date information
2. Always cite your sources with links to the websites you reference
3. Present information clearly and concisely
4. If information is time-sensitive, mention when the data was retrieved
5. Reply in English unless specifically asked for another language""",
    # 🔑 Key: Pass Bing tool definitions directly
    tools=bing_tool.definitions,
    description="Web Research Assistant with Grounding with Bing Search for retrieving real-time web information.",
    category="Research & Information",
    status="published",
    avatar_url="web-research-avatar.jpeg",
    function_list=[
        {"function_id": "bing-grounding-001", "function_name": "BingGrounding"}
    ],
    sample_prompts=[
        "What are the latest AI developments this week?",
        "What is the current weather in Seattle?",
        "What are the top tech news headlines today?",
        "Find recent articles about Azure AI services"
    ],
    scenarios=[
        {"title": "Research", "color": "primary"},
        {"title": "Current Events", "color": "info"}
    ],
    maintainers=[
        {
            "photoURL": "6acb666a-2cc7-49ac-ac9b-f4bb28cbbeea.jpg",
            "email": "demo@microsoft.com",
            "name": "Lei Xu",
            "permission": "view"
        }
    ]
)

print(f"✅ Created Bing-enabled agent!")
print(f"   Agent ID: {bing_agent.id}")
print(f"   Agent Name: {bing_agent.name}")
print(f"   Bing Grounding: ✓ Enabled")

---

## Part 5: Test the Agent

Let's test the agent with queries that require real-time web information.

### Run a Simple Query

In [ ]:
# Create a thread for conversation
bing_thread = agent_manager.create_thread()

print("💬 Created conversation thread")
print(f"   Thread ID: {bing_thread.id}")

# Ask a question that requires current web information
print("\n👤 User Query: What are the top 3 AI news stories today?")
print("\n⏳ Running agent with Bing Grounding...\n")

response = agent_manager.run_agent_simple(
    thread_id=bing_thread.id,
    agent_id=bing_agent.id,
    user_message="What are the top 3 AI news stories today?"
)

print("=" * 80)
print("🤖 Agent Response:")
print("=" * 80)
print(response)
print("=" * 80)

print("\n💡 Note: Response includes citations and source links from Bing search")

---

## Part 6: Display Citations (Bing Requirements)

According to [Bing's Use and Display Requirements](https://www.microsoft.com/en-us/bing/apis/grounding-legal#use-and-display-requirements), you must display:

1. **Website URLs** (from message annotations)
2. **Bing search query URLs** (from run step details)

Let's extract and display these citations properly.

In [ ]:
# Get messages from the thread
messages = agent_manager.get_messages(bing_thread.id)

print("=" * 80)
print("📎 EXTRACTING CITATIONS")
print("=" * 80)

for msg in messages:
    if msg.role == "assistant" and msg.text_messages:
        for text_msg in msg.text_messages:
            # Display the message content
            print(f"\n💬 Assistant Message:")
            print(f"{text_msg.text.value[:200]}..." if len(text_msg.text.value) > 200 else text_msg.text.value)
            
            # Display annotations (website URLs)
            if hasattr(text_msg.text, 'annotations') and text_msg.text.annotations:
                print(f"\n📌 Sources ({len(text_msg.text.annotations)} citations):")
                for i, annotation in enumerate(text_msg.text.annotations, 1):
                    if hasattr(annotation, 'url'):
                        print(f"  [{i}] {annotation.url}")
                    elif hasattr(annotation, 'file_citation'):
                        print(f"  [{i}] File: {annotation.file_citation}")
            else:
                print("\n⚠️  No annotations found in this message")

print("\n" + "=" * 80)
print("💡 IMPORTANT: Bing Requirements")
print("=" * 80)
print("In production applications, you MUST also display:")
print("  1. Bing search query URL: https://www.bing.com/search?q={query}")
print("  2. Bing logo/attribution")
print("  3. All source URLs from annotations")
print("\nSee: https://www.microsoft.com/en-us/bing/apis/grounding-legal")
print("=" * 80)

---

## Part 7: Additional Examples

### Example 1: Different Query Types

In [ ]:
# Test with different types of queries
test_queries = [
    "What's the current weather in Seattle?",
    "Find the latest Azure OpenAI announcements",
    "What are Microsoft's recent AI initiatives?"
]

print("=" * 80)
print("🧪 TESTING MULTIPLE QUERY TYPES")
print("=" * 80)

for i, query in enumerate(test_queries, 1):
    print(f"\n[Query {i}] {query}")
    print("-" * 80)
    
    response = agent_manager.run_agent_simple(
        thread_id=bing_thread.id,
        agent_id=bing_agent.id,
        user_message=query
    )
    
    # Display response (truncated)
    print(f"Response: {response[:150]}..." if len(response) > 150 else f"Response: {response}")
    print()

print("=" * 80)
print("✅ Completed multiple query tests")
print("=" * 80)

---

## Part 8: Cleanup

Clean up resources to avoid unnecessary costs.

In [ ]:
# Delete the agent and thread
agent_manager.delete_agent(bing_agent.id)
agent_manager.delete_thread(bing_thread.id)

print("✅ Cleanup Complete")
print("   • Agent deleted")
print("   • Thread deleted")

---

## Summary and Best Practices

### What We Learned

1. **Simple Integration**: Bing Grounding requires just 3 steps:
   - Import `BingGroundingTool`
   - Initialize with connection ID
   - Pass `tool.definitions` to agent

2. **Automatic Execution**: Unlike custom functions, Azure handles all tool calls automatically

3. **Built-in Citations**: Responses include source links and annotations

4. **Production Ready**: Managed by Azure infrastructure, no manual execution loop needed

### Best Practices

✅ **Use for Real-time Data**: Ideal for current events, news, weather, and live information  
✅ **Display Citations**: Always show source URLs per Bing's requirements  
✅ **Clear Instructions**: Tell the agent when to use Bing search  
✅ **Test Thoroughly**: Verify responses include proper citations  
✅ **Monitor Usage**: Track Bing API calls for cost management  

### When to Use Bing Grounding

**✅ Good Use Cases:**
- Research assistants
- News aggregators
- Customer support with product lookups
- Data gathering and analysis
- Real-time information queries

**❌ Not Ideal For:**
- Static/historical data (use knowledge bases)
- Private/internal data (use custom tools)
- Highly specific domain knowledge (use RAG)
- Offline scenarios (requires internet)

### Key Differences from Custom Functions

| Feature | Bing Grounding | Custom Functions |
|---------|---------------|------------------|
| Setup Complexity | Low (SDK class) | Medium (write functions) |
| Execution | Automatic by Azure | Manual by you |
| Maintenance | Managed by Azure | Your responsibility |
| Use Case | Web search | Custom logic |
| Tool Loop | Not needed | Required |

### Next Steps

1. **Explore More Tools**: Try Azure AI Search, Code Interpreter
2. **Combine Tools**: Use Bing + custom functions together
3. **Build Custom Tools**: Create domain-specific integrations
4. **Production Deployment**: Add monitoring, logging, error handling
5. **Cost Optimization**: Implement caching and query optimization

### Related Notebooks

- `02_mcp_integration.ipynb`: Model Context Protocol integration
- `../01_agent/`: Basic agent creation patterns
- `../03_monitoring/`: Agent tracing and observability
- `../05_evaluation/`: Agent evaluation and testing

### Additional Resources

- [Bing Grounding Documentation](https://learn.microsoft.com/azure/ai-services/openai/how-to/grounding)
- [Bing API Terms of Use](https://www.microsoft.com/en-us/bing/apis/grounding-legal)
- [Azure AI Foundry Tools](https://learn.microsoft.com/azure/ai-studio/how-to/use-tools)
- [Agent Best Practices](https://learn.microsoft.com/azure/ai-studio/concepts/agents)